<a href="https://colab.research.google.com/github/minami1009/bio/blob/main/openmm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#最初にminicondaをインストール
!wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
!rm Miniconda3-latest-Linux-x86_64.sh

In [ ]:
#OpenMMをconda-forgeチャンネルからインストール
!conda install -c conda-forge openmm=7.6 python=3.7 -y 

#OpenMMが使えるようにパスを通しておく
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
from google.colab import files

In [ ]:
#openmm-setupで準備したpythonプログラムを実行
%run run_openmm_simulation.py

In [ ]:
#できあがったファイルをとにかくダウンロードしておく
files.download("trajectory.dcd")
files.download("log.txt")
files.download("checkpoint.chk")
files.download("system.xml")
files.download("final_state.pdbx")

In [4]:
#ここからpytrajでtrajectoryファイルを見る
!conda install -c ambermd pytraj  --yes 2>&1 1>/dev/null

import pandas as pd
import matplotlib.pyplot as plt
import pytraj as pt
import numpy as np
import os

workDir = '/content/drive/MyDrive/'

#ローカルで作ったtrajectoryファイルを読み込むならここに入れる
#pt.iterloadの引数は1つ目がtrajectoryファイル(.dcd)、2つ目が初期状態のpdbファイル
#上で作ったtrajectoryファイルの場合は適当に1つ目の引数を変える
traj = pt.iterload('/content/drive/Othercomputers/MacBook_home/modelling/openmm/300ns/trajectory.dcd','2z70-processed.pdb')
print(traj)

In [ ]:
rmsd = pt.rmsd(traj, ref = 0, mask = "@CA")

#ここの2つの0.2は適宜変える
#今回は0.2nsごとに1500回やっているのでxlimはmax300にしている
time = len(rmsd)*0.2
time_array = np.arange(0,time,0.2)

ax = plt.plot(time_array, rmsd, alpha=0.6, color = 'blue', linewidth = 1.0)
plt.xlim(0, 300)
plt.ylim(0)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("RMSD [$\AA$]", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, "rmsd.png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(rmsd)
raw_data.to_csv(os.path.join(workDir, "rmsd.csv"))